<a href="https://colab.research.google.com/github/AbhayMudgal/Breast_Cancer_Detection/blob/main/B_C_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
import os, json
import numpy as np
import pandas as pd
from shutil import copyfile
from os import makedirs, listdir
import cv2
import random

In [ ]:
kaggle_json = {"username":"abhay1010","key":"4af3df22394b3bfc76ea750845f8b173"}
makedirs("/content/.kaggle/", exist_ok = True)
makedirs("/root/.kaggle/", exist_ok = True)
with open("/content/.kaggle/kaggle.json", 'w') as file:
  json.dump(kaggle_json, file)

!cp "/content/.kaggle/kaggle.json" "/root/.kaggle/kaggle.json"
!chmod 600 "/content/.kaggle/kaggle.json"
!chmod 600 "/root/.kaggle/kaggle.json"

In [ ]:
%ls "/content/.kaggle/"

kaggle.json


In [ ]:
! kaggle datasets list -s Histopathology

ref                                                            title                                             size  lastUpdated          downloadCount  
-------------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
paultimothymooney/breast-histopathology-images                 Breast Histopathology Images                       3GB  2017-12-19 05:46:40          27156  
andrewmvd/lung-and-colon-cancer-histopathological-images       Lung and Colon Cancer Histopathological Images     2GB  2020-04-14 06:57:56           1598  
forderation/breakhis-400x                                      BreaKHis 400X                                    803MB  2020-07-29 03:58:30            744  
kmader/skin-cancer-mnist-ham10000                              Skin Cancer MNIST: HAM10000                        5GB  2018-09-20 20:36:13          42983  
linjustin/train-val-test-tcga-coad-msi-mss                     G

In [ ]:
! kaggle datasets download -d paultimothymooney/breast-histopathology-images

100% 3.09G/3.10G [00:50<00:00, 57.3MB/s]
100% 3.10G/3.10G [00:51<00:00, 65.0MB/s]


In [ ]:
! unzip /content/breast-histopathology-images.zip

In [ ]:
size = 50
val_split = 0.1
Batch_size = 64
lr = 0.001
epochs = 20

In [ ]:
test = []
train = []
test_y = []
train_y = []
for files in listdir("/content/IDC_regular_ps50_idx5"):
  path = "/content/IDC_regular_ps50_idx5/" + files
  for types in listdir(path):
    label = int (types)
    dir = "/content/IDC_regular_ps50_idx5/" + files + "/" + types
    for sample in listdir(dir):
      image = dir + '/' + sample
      img = cv2.imread(image)
     
      img = cv2.resize(img, (size, size))
      
      data = np.asarray(img)
      
      if random.random()<val_split:
        test.append(data)
        test_y.append(label)
      else:
        train.append(data)  
        train_y.append(label)

In [1]:
import random
import tensorflow as tf
from tensorflow import keras as k
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(train, train_y, test_size = 0.15, random_state = 1)

X_train = np.array(X_train)
X_val = np.array(X_val)

print(X_train.shape, X_val.shape)

datagen = ImageDataGenerator(rescale = 1/255.0, width_shift_range=0.1, height_shift_range = 0.1, zoom_range = 0.1, shear_range = 0.1, fill_mode = "nearest")
val_datagen = ImageDataGenerator(rescale = 1/255.0, width_shift_range=0.1, height_shift_range = 0.1, zoom_range = 0.1)

train_gen = datagen.flow(X_train, Y_train, batch_size = Batch_size)
val_gen = val_datagen.flow(X_val, Y_val, batch_size = Batch_size)

(212467, 50, 50, 3) (37495, 50, 50, 3)


In [ ]:
CNN = k.Sequential([
  Conv2D(32, (3, 3), activation = 'relu', input_shape = (size, size, 3)),
  MaxPooling2D(2, 2),
  Dropout(0.2),
  Conv2D(64, (3, 3), activation = 'relu'),
  MaxPooling2D(2, 2),
  Dropout(0.2),
  Conv2D(128, (3, 3), activation = 'relu'),
  MaxPooling2D(2, 2),
  Dropout(0.2),
  Flatten(),
  Dense(512, activation = 'relu'),
  Dropout(0.2),
  Dense(64, activation = 'relu'),
  Dense(1, activation = 'sigmoid')                       
])

In [ ]:
opt = k.optimizers.Adam(learning_rate=0.01)
CNN.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
CNN.fit(train_gen, epochs = epochs, validation_data = val_gen)

In [ ]:
from keras.applications.vgg16 import VGG16

pre = VGG16(include_top = False, weights = 'imagenet', input_shape = (size, size, 3))

for layer in pre.layers:
  layer.trainable = False

x = Flatten()(pre.layers[-1].output)
x = Dense(512, activation = 'relu')(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(32, activation = 'relu')(x)
output = Dense(1, activation = 'sigmoid')(x)

VGG = keras.Model(pre.inputs, output)

VGG.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [accuracy])

run = VGG.fit(train_gen, epochs = 15, validation_data = val_gen)